In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time

In [ ]:
path = r'C:\Users\ken25\OneDrive\Desktop\01_Python\Set-Scraping-for-friends\chromedriver.exe' # เปลี่ยน path ให้เป็นของเครื่องตัวเอง
service = Service(path)

driver = webdriver.Chrome(service=service)

In [ ]:
df = pd.read_excel('mai_tech.xlsx',index_col=1)  # <- เปลี่ยนชื่อไฟล์ตรงนี้
stocks = df.index.to_list()
stocks

['APP',
 'BBIK',
 'BE8',
 'BVG',
 'COMAN',
 'I2',
 'IIG',
 'IRCP',
 'ITNS',
 'NAT',
 'NETBAY',
 'PLANET',
 'PROEN',
 'READY',
 'SECURE',
 'SICT',
 'SIMAT',
 'SPVI',
 'SRS',
 'TBN',
 'TERA',
 'TPS',
 'VCOM']

In [ ]:
base_url = 'https://www.settrade.com/th/equities/quote/{}/historical-trading'

# Initialize an empty dictionary to store the dataframes

all_data =[]

# Loop through each stock symbol
for stock in stocks:
    # Construct the URL
    url = base_url.format(stock)
    driver.get(url)
    time.sleep(15)
    driver.execute_script("window.scrollBy(0, 1400)")

    time.sleep(7)
    elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "multiselect__tags")))
    time.sleep(7)
    elements[1].click()

    
    elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "multiselect__element")))
    time.sleep(10)
    elements[-1].click()

    time.sleep(7)
    
    
    # Get page source
    data = driver.page_source
    
    # Parse HTML table into DataFrame
    data_df = pd.read_html(data)[1]
    
    # Clean column names
    clean_column = []
    for c in data_df.columns:
        clean_column.append(c.replace('  (Click to sort Ascending)', ''))
    data_df.columns = clean_column
    
    # Set index to 'วันที่'
    data_df = data_df.set_index('วันที่')
    
    # Filter rows and columns 
    dates = ['31 ต.ค. 2567'] #        <----       <----      <---            *******************************************เปลี่ยนวันที่ตรงนี้**********************************************************
    data_df = data_df.loc[data_df.index.intersection(dates), ['ราคาปิด']]
    #data_df = data_df.loc[dates, ['ราคาปิด']]
    
    # Transpose DataFrame and replace the first column name with the stock symbol
    data_df = data_df.transpose()
    data_df.index = [stock]
    

    
    all_data.append(data_df)

# Close the driver
driver.quit()

# Concatenate all DataFrames
combined_data = pd.concat(all_data)

# Write the combined DataFrame to a single sheet in an Excel file
combined_data.to_excel('stock_data_mai_tech-09.xlsx', sheet_name='All Stocks')

print("Data has been written")


C:\Users\ken25\AppData\Local\Temp\ipykernel_29360\486533581.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_df = pd.read_html(data)[1]


KeyboardInterrupt: 